In [ ]:
# Standard library imports
from collections import Counter
from logging import Logger
from functools import partial
from typing import List, Dict, Tuple, Optional, Callable, Any
import re
import random
import asyncio
# Third-party library imports
import torch
import numpy as np
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Local utility imports
from doraemon import Doraemon

class RelaxedFDA:
    """
    Author: -
    Date: 2025-06-08
    """

    PATTERN = r'\\boxed{([^}]+)}'

    @classmethod
    def get_encoder(cls, encoder_path: str = '/kaggle/input/encoder-l6-v2/transformers/v0.1.4/1') -> SentenceTransformer:
        return SentenceTransformer(encoder_path).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    @classmethod
    def create_embedding(cls, rs: str, encoder: SentenceTransformer) -> List[float]:
        return encoder.encode(rs, convert_to_tensor=False, show_progress_bar=False)

    @classmethod
    def get_answer(cls, raw_answer: str) -> str:
        match = re.search(cls.PATTERN, raw_answer, flags=re.IGNORECASE)
        if match:
            try:
                return match.group(1).strip()
            except IndexError:
                pass
        return str(raw_answer).strip()

    @classmethod
    def optimize_rs(cls, prompt: str, optimized_sys_prompt: str, logger: Logger, max_tokens:int=512) -> Tuple[str, str]:
        try:
            messages = [
                {"role": "system", "content": optimized_sys_prompt},
                {"role": "user", "content": prompt},
            ]
            improved_rs, tokens = Doraemon.inference(logger=logger, messages=messages, temperature=1.0, max_tokens=max_tokens)
            answer_match = cls.get_answer(improved_rs)
        except Exception as e:
            logger.exception(f"Error while querying LLM: {e}")
            improved_rs = None
            answer_match = None
        return improved_rs, answer_match

    @classmethod
    def prepare_dataset(
        cls,
        file_path: str,
        mode: str = 'r_s',
        logger=None,
        number_instance: Optional[int] = None,
        context: bool = False
    ) -> pd.DataFrame:
        if logger is None:
            raise ValueError("Logger must be provided")
        try:
            df = pd.read_pickle(file_path)
            logger.info(f"Loaded dataset from {file_path} with shape {df.shape}")
        except Exception as e:
            logger.exception(f"Error loading dataset from {file_path}: {e}")
            raise
        logger.info(f"In {mode} mode.")
        if context:
            grouped = (
                df.groupby('query', sort=False)
                .agg(r_s=(mode, list), context=('context', 'first'), g_t=('g_t', 'first'), answer=('answer', list))
                .reset_index()
            )
        else:
            grouped = (
                df.groupby('query', sort=False)
                .agg(r_s=(mode, list), g_t=('g_t', 'first'), answer=('answer', list))
                .reset_index()
            )
        if number_instance:
            grouped = grouped.head(number_instance)
            logger.info(f"Processed first {number_instance} instances.")
        logger.info(f"Dataset preparation completed with {grouped.shape[0]} entries.")
        return grouped

    @classmethod
    def get_optimize_system_prompt(cls, data_name: str) -> str:
        task_mapping = {
            "GSM8K": "mathematical reasoning",
            "MATH": "mathematical reasoning",
            "CommonsenseQA": "commonsense reasoning",
            "StrategyQA": "commonsense reasoning",
            "HotpotQA": "multihop reasoning",
            "MuSiQue": "multihop reasoning",
            "FEVER": "fact checking"
        }
        task_type = task_mapping.get(data_name, "reasoning")
        OPTIMIZE_SYSTEM_PROMPT = (
            f"You are a concise and helpful assistant for {task_type} tasks. "
            "Provide step-by-step solutions, ensuring accuracy. "
            "Conclude with the final answer in the format: \\boxed{answer}. "
            "When given a reasoning process, refine it to be both correct and succinct."
        )
        return OPTIMIZE_SYSTEM_PROMPT

    @classmethod
    def causal_infer_with_fdr(
        cls,
        question: str,
        context: Any,
        logger: Logger,
        optimize_sys_prompt: str,
        encoder: SentenceTransformer,
        r_s: List[str],
        D: List[Dict],
        enable_logger_rs: bool = True,
        nwgm_mode: Optional[str] = None,
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512
    ) -> Tuple[str, float]:
        raw_rs = r_s
        # Create embeddings
        raw_rs_embedding = []
        for rs in raw_rs:
            try:
                embd = cls.create_embedding(rs, encoder)
                if embd is None or len(embd) == 0:
                    logger.warning(f"Empty or None embedding for reasoning path.")
                    continue
                raw_rs_embedding.append(embd)
            except Exception as e:
                logger.exception(f"Error generating embedding: {str(e)}")
                raw_rs_embedding.append(None)
        # filter out None values and ensure alignment
        valid_pairs = [(rs, emb) for rs, emb in zip(raw_rs, raw_rs_embedding) if rs is not None and emb is not None]
        if not valid_pairs or len(valid_pairs) < K:
            logger.exception(f"Not enough valid reasoning paths for clustering: {len(valid_pairs)} found, {K} required. Skipping.")
            return None, 0.0
        raw_rs, raw_embeddings = zip(*valid_pairs)
        raw_rs = list(raw_rs)
        raw_rs_embedding = np.array(raw_embeddings)
        if enable_logger_rs:
            logger.info(f"Question: {question}")
            logger.info(f"Context: {context if context is not None else '[No context]'}")

        logger.info(f"Generated {len(raw_rs)} valid reasoning path and embeddings.")
        # Cluster embeddings
        kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(raw_rs_embedding)
        cluster_centers = kmeans.cluster_centers_
        all_cluster_results = []
        for cluster_idx in range(K):
            indices = np.where(cluster_labels == cluster_idx)[0]
            if len(indices) == 0:
                logger.info("Cluster %d: No items found", cluster_idx + 1)
                continue
            prob_rk_input = len(indices) / raw_rs_embedding.shape[0]

            # Find representative reasoning path (closest to cluster center)
            cluster_embeddings = raw_rs_embedding[indices]
            distances = np.linalg.norm(cluster_embeddings - cluster_centers[cluster_idx], axis=1)
            rep_index = indices[np.argmin(distances)]
            rep_text = raw_rs[rep_index]
            rk_emb = cls.create_embedding(rep_text, encoder)

            if enable_logger_rs:
                logger.info(
                    "Cluster %d: |C_%d| = %d, P(r_%d|do(X)) = %d/%d = %.3f\nRepresentative reasoning path (r_%d): %s",
                    cluster_idx + 1,
                    cluster_idx,
                    len(indices),
                    cluster_idx,
                    len(indices),
                    raw_rs_embedding.shape[0],
                    prob_rk_input,
                    cluster_idx,
                    rep_text
                )
            else:
                logger.info(
                    "Cluster %d: |C_%d| = %d, P(r_%d|do(X)) = %d/%d = %.3f",
                    cluster_idx + 1,
                    cluster_idx,
                    len(indices),
                    cluster_idx,
                    len(indices),
                    raw_rs_embedding.shape[0],
                    prob_rk_input
                )
            
            # Select top demonstrations based on similarity
            demos_scores = []
            for d_j in D:
                r_wrong_rs = d_j['wrong_rs']
                r_wrong_embed = cls.create_embedding(r_wrong_rs, encoder)
                sim = cosine_similarity([rk_emb], [r_wrong_embed])[0][0]
                demos_scores.append((d_j, sim))
                
            if nwgm_mode == 'reverse':
                sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=False)
            elif nwgm_mode == 'random':
                sorted_demos = [random.choice(demos_scores)]
            else:
                sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=True)

            top_demos = [i[0] for i in sorted_demos[:T]]  # Top 3 demos
            # Construct intervention prompt based on the question
            p_iter_rk = cls.construct_interven_prompt(
                selected_demos=top_demos,
                r_k=rep_text,
                test_question=question,
                test_context=context,
                has_context=bool(context)
            )
            if enable_logger_rs:
                logger.info(f"Has context is {bool(context)}")
                logger.info(f"The construct intervention prompt template is\n {p_iter_rk}")

            # Optimize reasoning paths
            improved_rss = []
            answers = []
            for _ in range(T):
                try:
                    improved_rs, final_ans = cls.optimize_rs(p_iter_rk, optimize_sys_prompt, logger, max_tokens)
                    if enable_logger_rs:
                        logger.info(f"Improved reasoning paths: \n{improved_rs}\nFinal answer: \n{final_ans}")  # <-- Logging both
                    improved_rss.append(improved_rs)
                    answers.append(final_ans)
                except Exception as e:
                    logger.exception(f"Error optimizing CoT for cluster {cluster_idx + 1}: {str(e)}")
            # Determine best answer
            answer_counts = Counter(answers)
            if answer_counts:
                best_answer, best_answer_count = max(answer_counts.items(), key=lambda x: x[1])
                prob_a_rk = best_answer_count / T
                if enable_logger_rs:
                    logger.info(
                        "P(A = %s | do(r_%d)) = %d/%d = %.3f",
                        str(best_answer),
                        cluster_idx,
                        best_answer_count,
                        T,
                        prob_a_rk
                    )
            else:
                logger.warning(f"No valid answers for cluster {cluster_idx + 1}")
                continue

            all_cluster_results.append({
                'r_k': rep_text,
                'answers': answers,
                'final_answer': best_answer,
                'prob_rk_input': prob_rk_input,
                'prob_a_rk': prob_a_rk
            })

        # correct location
        aggregated_votes = {}
        candidate_weights = {}
        for res in all_cluster_results:
            weight = res['prob_rk_input'] * res['prob_a_rk']
            candidate = res['final_answer']
            aggregated_votes[candidate] = aggregated_votes.get(candidate, 0) + weight
            candidate_weights[candidate] = candidate_weights.get(candidate, 0) + weight
        if not aggregated_votes:
            logger.exception("No aggregated votes generated")
            return None, 0.0
        # Select final answer
        final_answer, final_weight = max(aggregated_votes.items(), key=lambda x: x[1])
        overall_probability = final_weight / candidate_weights[final_answer] if candidate_weights[final_answer] > 0 else 0
        if enable_logger_rs:
            # Log the weighted voting calculation process
            calc_details = {}
            for res in all_cluster_results:
                candidate = res['final_answer']
                term = f"{res['prob_rk_input']:.3f} * {res['prob_a_rk']:.3f}"
                if candidate not in calc_details:
                    calc_details[candidate] = []
                calc_details[candidate].append(term)
        
            calc_strings = []
            for candidate, terms in calc_details.items():
                eq_str = " + ".join(terms)
                total = aggregated_votes[candidate]
                calc_strings.append(
                    f"P(A = {candidate}|do(X)) = {eq_str} = {total:.4f}"
                )
        
            # Log detailed calculation and final answer
            logger.info(
                "\nFinal results\n"
                "The final answer is obtained by performing a weighted voting as follows:\n"
                + "\n".join(calc_strings) +
                f"\nFinally, we chose the answer with the largest weight as the final answer.\n"
                f"Therefore, the final answer obtained according to the Relaxed-FDA is {final_answer}."
            )
        else:
            logger.info(
                "Aggregated votes (weighted): %s | Final answer: %s (weight: %s) | Overall prob: %.2f",
                aggregated_votes, final_answer, final_weight, overall_probability
            )

        for handler in logger.handlers:
            handler.flush()
        return final_answer, overall_probability



    @classmethod
    def construct_interven_prompt(
        cls,
        selected_demos: List[Dict],
        r_k: str,
        test_question: str,
        test_context: Any,
        has_context: bool = False
    ) -> str:
        prompt = ""
        for i, demo in enumerate(selected_demos, start=1):
            prompt += f"Demo {i}:\n"
            prompt += f"Q: The question is: {demo['question']}\n"
            if has_context:
                prompt += f"E: The context is: {demo['context']}\n"
            prompt += f"The provided reasoning path is: {demo['wrong_rs']}\n"
            prompt += f"A: The improved reasoning path is: {demo['correct_rs']}\n"
            prompt += f"Therefore, the correct answer is: {RelaxedFDA.get_answer(demo['correct_rs'])}\n\n"
        prompt += "Test Example:\n"
        prompt += f"Q: The question is: {test_question}\n"
        if has_context:
            prompt += f"E: The context is: {test_context}\n"
        prompt += "<think>/nLet us think step by step.\n"
        prompt += f"The provided reasoning path is: {r_k}\n"
        prompt += "A: The improved reasoning path is: [improved_rs]\n"
        prompt += "Therefore, the correct answer is: [answer]\n"
        return prompt

    @classmethod
    def evaluate_row(
        cls,
        row: Dict[str, Any],
        logger: Logger,
        op_system_prompt,
        encoder,
        D,
        enable_logger_rs:bool,
        ablation: str = 'all',
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512
    ) -> Dict[str, Any]:
        try:
            q = row.get('query')
            r_p = row.get('r_s')
            context = row.get('context') if row.get('context') else None

            if ablation in ['all', 'reverse', 'random']:
                final_answer, o_p = cls.causal_infer_with_fdr(
                    q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs, ablation, K, T, max_tokens
                )
            elif ablation == 'wo_wt':
                final_answer, o_p = cls.causal_infer_with_fdr_kmeans_wo_weight(
                    q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs
                )
            elif ablation == 'wo_km':
                final_answer, o_p = cls.causal_infer_with_fdr_no_kmeans_cluster(
                    q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs
                )
            else:
                raise ValueError(f"Unknown mode: {ablation}")

            
            if final_answer is not None:
                try:
                    final_answer_clean = cls.get_answer(final_answer)
                except Exception as e:
                    logger.exception(f"Exception in get_answer for question: {q}, error: {e}")
                    final_answer_clean = None
            else:
                final_answer_clean = None
            row = row.copy()
            row['final_answer'] = final_answer_clean
            row['o_p'] = o_p
            return row
        except Exception as e:
            logger.exception(f"Exception in evaluate_row for question: {row.get('question')}, error: {e}")
            row = row.copy()
            row['final_answer'] = None
            row['o_p'] = None
            return row

    @classmethod
    def evaluate(
        cls,
        grouped_data: pd.DataFrame,
        logger,
        op_system_prompt: str,
        encoder,
        D,
        enable_logger_rs=True,
        max_workers: int = 1,
        ablation: str = "all",
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512
    ) -> pd.DataFrame:
        from concurrent.futures import ProcessPoolExecutor
        from functools import partial
        from tqdm import tqdm
        
        tasks = grouped_data.to_dict(orient='records')
        if enable_logger_rs:
            logger.info(f"The total tasks: {len(tasks)}")
        process_func = partial(
            cls.evaluate_row,
            logger=logger,
            op_system_prompt=op_system_prompt,
            encoder=encoder,
            D=D,
            enable_logger_rs=enable_logger_rs,
            ablation=ablation,
            K=K,
            T=T,
            max_tokens=max_tokens
        )
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            results = list(tqdm(executor.map(process_func, tasks), total=len(tasks)))
        filtered_results = [r for r in results if r is not None]
        return pd.DataFrame(filtered_results)

    @classmethod
    def causal_infer_with_fdr_kmeans_wo_weight(
        cls,
        question: str,
        context: Any,
        logger,
        optimize_sys_prompt: str,
        encoder,
        r_s: List[str],
        D: List[dict],
        enable_logger_rs: bool = True,
        K: int = 4,
        T: int = 3,
    ) -> Tuple[str, float]:
        """
        Ablation: KMeans clustering, no weighted aggregation.
    
        This variant applies KMeans clustering to reasoning path embeddings.
        For each cluster, it selects a representative reasoning path, performs T optimizations,
        and uses simple majority voting among the T answers to pick the cluster's final answer.
    
        For final aggregation, all clusters' final answers are combined by **simple majority vote** 
        (not weighted by cluster size or answer probability). The most frequent answer across clusters
        is returned as the final answer, along with its proportion as an estimated probability.
    
        Args:
            question: The target question.
            context: Additional context for the question.
            logger: Logger instance.
            optimize_sys_prompt: Prompt for optimization.
            encoder: Embedding model.
            r_s: List of reasoning paths.
            D: Demonstration data.
            enable_logger_rs: Log intermediate results.
            K: Number of clusters for KMeans.
            T: Number of optimizations per cluster.
    
        Returns:
            final_answer: The aggregated answer.
            overall_probability: Proportion of clusters supporting the final answer.
        """
        raw_rs_embedding = []
        for rs in r_s:
            try:
                embd = cls.create_embedding(rs, encoder)
                if embd is not None and len(embd) > 0:
                    raw_rs_embedding.append((rs, embd))
            except Exception as e:
                logger.exception(f"Error generating embedding: {str(e)}")
        if not raw_rs_embedding or len(raw_rs_embedding) < K:
            logger.exception("Not enough valid reasoning paths for clustering.")
            return None, 0.0
        raw_rs, raw_embeddings = zip(*raw_rs_embedding)
        raw_rs = list(raw_rs)
        raw_rs_embedding = np.array(raw_embeddings)
    
        # Step 2: Clustering
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(raw_rs_embedding)
        cluster_centers = kmeans.cluster_centers_
    
        # Step 3: For each cluster, get best answer by majority (no weighting)
        all_cluster_results = []
        for cluster_idx in range(K):
            indices = np.where(cluster_labels == cluster_idx)[0]
            if len(indices) == 0:
                continue
            cluster_embeddings = raw_rs_embedding[indices]
            distances = np.linalg.norm(cluster_embeddings - cluster_centers[cluster_idx], axis=1)
            rep_index = indices[np.argmin(distances)]
            rep_text = raw_rs[rep_index]
            rk_emb = cls.create_embedding(rep_text, encoder)
    
            demos_scores = []
            for d_j in D:
                r_wrong_rs = d_j['wrong_rs']
                r_wrong_embed = cls.create_embedding(r_wrong_rs, encoder)
                sim = 0
                if r_wrong_embed is not None:
                    sim = np.dot(rk_emb, r_wrong_embed) / (np.linalg.norm(rk_emb) * np.linalg.norm(r_wrong_embed))
                demos_scores.append((d_j, sim))
            sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=True)
            top_demos = [i[0] for i in sorted_demos[:3]]
            p_iter_rk = cls.construct_interven_prompt(
                selected_demos=top_demos,
                r_k=rep_text,
                test_question=question,
                test_context=context,
                has_context=bool(context)
            )
            answers = []
            for _ in range(T):
                try:
                    improved_rs, final_ans = cls.optimize_rs(p_iter_rk, optimize_sys_prompt, logger)
                    answers.append(final_ans)
                except Exception as e:
                    logger.exception(f"Error optimizing CoT for cluster {cluster_idx + 1}: {str(e)}")
            if answers:
                best_answer, _ = Counter(answers).most_common(1)[0]
                all_cluster_results.append(best_answer)
    
        if not all_cluster_results:
            logger.exception("No valid cluster results")
            return None, 0.0
        agg_counts = Counter(all_cluster_results)
        final_answer, vote_count = agg_counts.most_common(1)[0]
        overall_probability = vote_count / len(all_cluster_results)
        logger.info(f"Aggregated candidate votes (majority, with kmeans, no weight): {agg_counts}")
        return final_answer, overall_probability
    
    @classmethod
    def causal_infer_with_fdr_no_kmeans_cluster(
        cls,
        question: str,
        context: Any,
        logger,
        optimize_sys_prompt: str,
        encoder,
        r_s: List[str],
        D: List[dict],
        enable_logger_rs: bool = True,
        T: int = 3,
    ) -> Tuple[str, float]:
        """
        Ablation: No KMeans clustering, weighted aggregation.
    
        This variant **does not cluster** the reasoning paths. Instead, it treats each reasoning path independently.
        For each path, T optimizations are performed, and the majority answer and its frequency (as a probability) are recorded.
    
        In final aggregation, each path's best answer contributes to the overall answer **weighted by its answer frequency**.
        The answer with the highest summed weight across all paths is returned as the final answer, and its normalized weight
        is reported as the estimated probability.
    
        Args:
            question: The target question.
            context: Additional context for the question.
            logger: Logger instance.
            optimize_sys_prompt: Prompt for optimization.
            encoder: Embedding model.
            r_s: List of reasoning paths.
            D: Demonstration data.
            enable_logger_rs: Log intermediate results.
            T: Number of optimizations per reasoning path.
    
        Returns:
            final_answer: The aggregated answer.
            overall_probability: Weighted probability of the final answer.
        """
        
        raw_rs_embedding = []
        for rs in r_s:
            try:
                embd = cls.create_embedding(rs, encoder)
                if embd is not None and len(embd) > 0:
                    raw_rs_embedding.append((rs, embd))
            except Exception as e:
                logger.exception(f"Error generating embedding: {str(e)}")
        if not raw_rs_embedding:
            logger.exception("No valid reasoning paths.")
            return None, 0.0
    
        all_path_results = []
        for idx, (rep_text, rk_emb) in enumerate(raw_rs_embedding):
            demos_scores = []
            for d_j in D:
                r_wrong_rs = d_j['wrong_rs']
                r_wrong_embed = cls.create_embedding(r_wrong_rs, encoder)
                sim = 0
                if r_wrong_embed is not None:
                    sim = np.dot(rk_emb, r_wrong_embed) / (np.linalg.norm(rk_emb) * np.linalg.norm(r_wrong_embed))
                demos_scores.append((d_j, sim))
            sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=True)
            top_demos = [i[0] for i in sorted_demos[:3]]
            p_iter_rk = cls.construct_interven_prompt(
                selected_demos=top_demos,
                r_k=rep_text,
                test_question=question,
                test_context=context,
                has_context=bool(context)
            )
            answers = []
            for _ in range(T):
                try:
                    improved_rs, final_ans = cls.optimize_rs(p_iter_rk, optimize_sys_prompt, logger)
                    answers.append(final_ans)
                except Exception as e:
                    logger.exception(f"Error optimizing CoT for reasoning path {idx + 1}: {str(e)}")
            answer_counts = Counter(answers)
            if answer_counts:
                best_answer, best_answer_count = answer_counts.most_common(1)[0]
                prob_a_rk = best_answer_count / T
                all_path_results.append({'final_answer': best_answer, 'prob_a_rk': prob_a_rk})
    
        # Weighted aggregation
        aggregated_votes = {}
        total_weight = 0.0
        for res in all_path_results:
            weight = res['prob_a_rk']
            candidate = res['final_answer']
            aggregated_votes[candidate] = aggregated_votes.get(candidate, 0) + weight
            total_weight += weight
        if not aggregated_votes:
            logger.exception("No aggregated votes generated")
            return None, 0.0
        final_answer, final_weight = max(aggregated_votes.items(), key=lambda x: x[1])
        overall_probability = final_weight / total_weight if total_weight > 0 else 0
        logger.info(f"Aggregated candidate votes (weighted, no kmeans): {aggregated_votes}")
        return final_answer, overall_probability


    @classmethod
    async def async_optimize_rs(cls, prompt: str, optimized_sys_prompt: str, logger: Logger, max_tokens: int = 512, max_concurrent: int = 50) -> Tuple[str, str]:
        try:
            messages = [
                {"role": "system", "content": optimized_sys_prompt},
                {"role": "user", "content": prompt},
            ]
            results = await Doraemon.async_inference(
                logger=logger,
                prompts=[messages],
                temperatures=[1.0],
                max_tokens=max_tokens,
                max_concurrent=max_concurrent
            )
            improved_rs, tokens = results[0]
            answer_match = cls.get_answer(improved_rs)
        except Exception as e:
            logger.exception(f"Error while querying LLM: {e}")
            improved_rs = None
            answer_match = None
        return improved_rs, answer_match
    
    @classmethod
    async def async_create_embeddings(cls, raw_rs: List[str], encoder: SentenceTransformer, logger: Logger) -> np.ndarray:
        raw_rs_embedding = []
        for rs in raw_rs:
            try:
                embd = await cls.async_create_embedding(rs, encoder)  # Async version here
                if embd is None or len(embd) == 0:
                    logger.warning(f"Empty or None embedding for reasoning path.")
                    continue
                raw_rs_embedding.append(embd)
            except Exception as e:
                logger.exception(f"Error generating embedding: {str(e)}")
                raw_rs_embedding.append(None)
        return np.array([emb for emb in raw_rs_embedding if emb is not None])
    
    @classmethod
    async def async_create_embedding(cls, rs: str, encoder: SentenceTransformer) -> np.ndarray:
        """
        Perform async embedding generation.
        """
        loop = asyncio.get_event_loop()
        func = partial(encoder.encode, rs, show_progress_bar=False)
        embd = await loop.run_in_executor(None, func)
        return embd

    
    @classmethod
    async def async_cluster_embeddings(
        cls,
        raw_rs_embedding: np.ndarray,
        K: int,
        logger: Logger
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Perform asynchronous clustering on the embeddings.
        """
        loop = asyncio.get_event_loop()
        cluster_labels, cluster_centers = await loop.run_in_executor(
            None, 
            lambda: cls._perform_kmeans_clustering(raw_rs_embedding, K, logger)
        )
        return cluster_labels, cluster_centers
    
    @classmethod
    def _perform_kmeans_clustering(
        cls,
        raw_rs_embedding: np.ndarray,
        K: int,
        logger: Logger
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Helper method to perform KMeans clustering synchronously (called from an executor).
        """
        kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(raw_rs_embedding)
        cluster_centers = kmeans.cluster_centers_
        return cluster_labels, cluster_centers
    
    @classmethod
    async def async_process_clusters(
        cls,
        cluster_labels: np.ndarray,
        cluster_centers: np.ndarray,
        r_s: List[str],
        D: List[Dict],
        question: str,
        context: Any,
        logger: Logger,
        K: int,
        T: int,
        optimize_sys_prompt: str,
        enable_logger_rs: bool,
        nwgm_mode: Optional[str],
        max_tokens: int,
        raw_rs_embedding: np.ndarray,
        encoder,
        max_concurrent: int = 50
    ) -> List[Dict]:
        all_cluster_results = []
    
        # Iterate over clusters and process each cluster asynchronously
        for cluster_idx in range(K):
            indices = np.where(cluster_labels == cluster_idx)[0]
            if len(indices) == 0:
                logger.info(f"Cluster {cluster_idx + 1}: No items found")
                continue
            prob_rk_input = len(indices) / raw_rs_embedding.shape[0]
    
            # Find the representative reasoning path (closest to cluster center)
            cluster_embeddings = raw_rs_embedding[indices]
            distances = np.linalg.norm(cluster_embeddings - cluster_centers[cluster_idx], axis=1)
            rep_index = indices[np.argmin(distances)]
            rep_text = r_s[rep_index]
            rk_emb = raw_rs_embedding[rep_index]

            demos_scores = []
            for d_j in D:
                r_wrong_rs = d_j['wrong_rs']
                r_wrong_embed = await cls.async_create_embedding(r_wrong_rs, encoder)
                sim = 0
                if r_wrong_embed is not None:
                    sim = np.dot(rk_emb, r_wrong_embed) / (np.linalg.norm(rk_emb) * np.linalg.norm(r_wrong_embed))
                demos_scores.append((d_j, sim))
            sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=True)
            top_demos = [i[0] for i in sorted_demos[:3]]
    
            # Construct prompt using selected demos
            p_iter_rk = cls.construct_interven_prompt(
                selected_demos=top_demos,
                r_k=rep_text,
                test_question=question,
                test_context=context,
                has_context=bool(context)
            )

            # Optimize reasoning paths T times (async!)
            tasks = [cls.async_optimize_rs(p_iter_rk, optimize_sys_prompt, logger, max_tokens, max_concurrent) for _ in range(T)]
            improved_results = await asyncio.gather(*tasks)
            improved_rss = [improved_rs for improved_rs, _ in improved_results]
            answers = [final_ans for _, final_ans in improved_results]
    
            # Determine best answer
            answer_counts = Counter(answers)
            if answer_counts:
                best_answer, best_answer_count = answer_counts.most_common(1)[0]
                prob_a_rk = best_answer_count / T
            else:
                best_answer = None
                prob_a_rk = 0.0
    
            # Append the full cluster result
            all_cluster_results.append({
                'r_k': rep_text,
                'answers': answers,
                'final_answer': best_answer,
                'prob_rk_input': prob_rk_input,
                'prob_a_rk': prob_a_rk
            })
    
        return all_cluster_results
    
    @classmethod
    async def async_aggregate_votes(cls, all_cluster_results: List[Dict], logger: Logger) -> Tuple[str, float]:
        # Calculate final answer and probabilities
        aggregated_votes = {}
        candidate_weights = {}
        for res in all_cluster_results:
            weight = res['prob_rk_input'] * res['prob_a_rk']
            candidate = res['final_answer']
            aggregated_votes[candidate] = aggregated_votes.get(candidate, 0) + weight
            candidate_weights[candidate] = candidate_weights.get(candidate, 0) + weight
        
        if not aggregated_votes:
            logger.exception("No aggregated votes generated")
            return None, 0.0
        
        # Select final answer
        final_answer, final_weight = max(aggregated_votes.items(), key=lambda x: x[1])
        overall_probability = final_weight / candidate_weights[final_answer] if candidate_weights[final_answer] > 0 else 0
        
        return final_answer, overall_probability

    @classmethod
    async def async_causal_infer_with_fdr(
        cls,
        question: str,
        context: Any,
        logger: Logger,
        optimize_sys_prompt: str,
        encoder,
        r_s: List[str],
        D: List[Dict],
        enable_logger_rs: bool = True,
        nwgm_mode: Optional[str] = None,
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512,
        max_concurrent: int = 20
    ) -> Tuple[str, float]:

        # 1. Async create embeddings
        raw_rs_embedding = await cls.async_create_embeddings(r_s, encoder, logger)
        if len(raw_rs_embedding) < K:
            logger.exception(f"Not enough valid reasoning paths for clustering: {len(raw_rs_embedding)} found, {K} required. Skipping.")
            return None, 0.0

        # 2. Async cluster
        cluster_labels, cluster_centers = await cls.async_cluster_embeddings(raw_rs_embedding, K, logger)

        # 3. Async process clusters
        all_cluster_results = await cls.async_process_clusters(
            cluster_labels, cluster_centers, r_s, D, question, context,
            logger, K, T, optimize_sys_prompt, enable_logger_rs, nwgm_mode, max_tokens, raw_rs_embedding, encoder, max_concurrent
        )

        # 4. Aggregate
        return await cls.async_aggregate_votes(all_cluster_results, logger)

    @classmethod
    async def async_causal_infer_with_fdr_kmeans_wo_weight(
        cls,
        question: str,
        context: Any,
        logger,
        optimize_sys_prompt: str,
        encoder,
        r_s: List[str],
        D: List[dict],
        enable_logger_rs: bool = True,
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512,
        max_concurrent: int = 20
    ) -> Tuple[str, float]:
        """
        Async: KMeans clustering, no weighted aggregation.
        """
        # 1. Async create embeddings
        raw_rs_embedding = await cls.async_create_embeddings(r_s, encoder, logger)
        if len(raw_rs_embedding) < K:
            logger.exception(f"Not enough valid reasoning paths for clustering: {len(raw_rs_embedding)} found, {K} required. Skipping.")
            return None, 0.0
    
        # 2. Async cluster
        cluster_labels, cluster_centers = await cls.async_cluster_embeddings(raw_rs_embedding, K, logger)
    
        # 3. Async process clusters (reuse async_process_clusters, or create a variant if majority voting per-cluster is needed)
        all_cluster_results = await cls.async_process_clusters(
            cluster_labels, cluster_centers, r_s, D, question, context,
            logger, K, T, optimize_sys_prompt, enable_logger_rs, None, max_tokens, raw_rs_embedding, encoder, max_concurrent
        )
    
        # 4. Aggregate with majority vote, not weighted
        return await cls.async_aggregate_votes_majority(all_cluster_results, logger)
    
    @classmethod
    async def async_aggregate_votes_majority(cls, all_cluster_results: List[dict], logger) -> Tuple[str, float]:
        """
        Aggregates answers by simple majority vote (not weighted).
        """
        candidate_list = [res['final_answer'] for res in all_cluster_results if res['final_answer'] is not None]
        if not candidate_list:
            logger.exception("No valid cluster results for majority vote")
            return None, 0.0
    
        agg_counts = Counter(candidate_list)
        final_answer, vote_count = agg_counts.most_common(1)[0]
        overall_probability = vote_count / len(candidate_list)
        logger.info(f"Aggregated candidate votes (majority, with kmeans, no weight): {agg_counts}")
        return final_answer, overall_probability

    @classmethod
    async def async_causal_infer_with_fdr_no_kmeans_cluster(
        cls,
        question: str,
        context: Any,
        logger,
        optimize_sys_prompt: str,
        encoder,
        r_s: List[str],
        D: List[dict],
        enable_logger_rs: bool = True,
        T: int = 3,
        max_tokens: int = 512,
    ) -> Tuple[str, float]:
        # 1. Async create embeddings
        raw_rs_embedding = await cls.async_create_embeddings(r_s, encoder, logger)
        if len(raw_rs_embedding) == 0:
            logger.exception("No valid reasoning paths.")
            return None, 0.0
    
        # 2. Async process paths (per path, no clustering)
        all_path_results = await cls.async_process_paths(
            raw_rs_embedding, r_s, D, question, context,
            logger, encoder, T, optimize_sys_prompt, enable_logger_rs, max_tokens
        )
    
        # 3. Weighted aggregation (reuse async_aggregate_votes)
        return await cls.async_aggregate_votes(all_path_results, logger)
    
    
    @classmethod
    async def async_process_paths(
        cls,
        raw_rs_embedding: np.ndarray,
        r_s: List[str],
        D: List[dict],
        question: str,
        context: Any,
        logger,
        encoder,
        T: int,
        optimize_sys_prompt: str,
        enable_logger_rs: bool,
        max_tokens: int,
        max_concurrent: int = 50
    ) -> List[dict]:
        """
        For each reasoning path: select top demos, optimize T times (async), get best answer and its probability.
        Return a list of dicts like: {'final_answer': ..., 'prob_a_rk': ...}
        """
        all_path_results = []
        n_paths = len(r_s)
        for idx, (rep_text, rk_emb) in enumerate(zip(r_s, raw_rs_embedding)):
            demos_scores = []
            for d_j in D:
                r_wrong_rs = d_j['wrong_rs']
                r_wrong_embed = await cls.async_create_embedding(r_wrong_rs, encoder)
                sim = 0
                if r_wrong_embed is not None:
                    sim = np.dot(rk_emb, r_wrong_embed) / (np.linalg.norm(rk_emb) * np.linalg.norm(r_wrong_embed))
                demos_scores.append((d_j, sim))
            sorted_demos = sorted(demos_scores, key=lambda x: x[1], reverse=True)
            top_demos = [i[0] for i in sorted_demos[:3]]
            p_iter_rk = cls.construct_interven_prompt(
                selected_demos=top_demos,
                r_k=rep_text,
                test_question=question,
                test_context=context,
                has_context=bool(context)
            )
            tasks = [cls.async_optimize_rs(p_iter_rk, optimize_sys_prompt, logger, max_tokens, max_concurrent) for _ in range(T)]
            improved_results = await asyncio.gather(*tasks)
            answers = [final_ans for _, final_ans in improved_results if final_ans is not None]
            answer_counts = Counter(answers)
            if answer_counts:
                best_answer, best_answer_count = answer_counts.most_common(1)[0]
                prob_a_rk = best_answer_count / T
                all_path_results.append({'final_answer': best_answer, 'prob_a_rk': prob_a_rk, 'prob_rk_input': 1.0 / n_paths})
    
        return all_path_results

    @classmethod
    async def async_evaluate(
        cls,
        tasks: List[Dict[str, Any]],
        logger,
        op_system_prompt: str,
        encoder,
        D,
        enable_logger_rs=True,
        ablation: str = "all",
        K: int = 4,
        T: int = 3,
        max_tokens: int = 512,
        max_concurrent: int = 20,
        p_b: bool = True
    ) -> pd.DataFrame:
    
        async def evaluate_single_row(row):
            try:
                q = row.get('query')
                r_p = row.get('r_s')
                context = row.get('context') if row.get('context') else None

                if ablation in ['all', 'reverse', 'random']:
                    final_answer, o_p = await cls.async_causal_infer_with_fdr(
                        q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs, ablation, K, T, max_tokens, max_concurrent
                    )
                elif ablation == 'wo_km':
                    final_answer, o_p = await cls.async_causal_infer_with_fdr_no_kmeans_cluster(
                        q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs
                    )
                elif ablation == 'wo_wt':
                    final_answer, o_p = await cls.async_causal_infer_with_fdr_kmeans_wo_weight(
                        q, context, logger, op_system_prompt, encoder, r_p, D, enable_logger_rs, K, T, max_tokens, max_concurrent
                    )
                else:
                    raise ValueError(f"Unknown mode: {ablation}")
    
                if final_answer is not None:
                    try:
                        final_answer_clean = cls.get_answer(final_answer)
                    except Exception as e:
                        logger.exception(f"Exception in get_answer for question: {q}, error: {e}")
                        final_answer_clean = None
                else:
                    final_answer_clean = None
    
                row = row.copy()
                row['final_answer'] = final_answer_clean
                row['o_p'] = o_p
                return row
            except Exception as e:
                logger.exception(f"Exception in evaluate_row for question: {row.get('question')}, error: {e}")
                row = row.copy()
                row['final_answer'] = None
                row['o_p'] = None
                return row

        # Build all coroutines
        all_coros = [evaluate_single_row(row) for row in tasks]

        if p_b:
            results = await tqdm_asyncio.gather(
                *all_coros,
                total=len(all_coros),
                desc="Evaluating rows"
            )
        else:
            # 📊 Use tqdm_asyncio.gather for a high-level progress bar
            results = await asyncio.gather(*all_coros)
        
        filtered_results = [r for r in results if r is not None]
        return pd.DataFrame(filtered_results)

2025-07-25 07:02:35.741580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753426955.984326      14 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753426956.061694      14 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 10.7 MB/s eta 0:00:00
